#### 1. Download the college dataset for BigQuery:

In [11]:
%env BASE_DIR=/home/jupyter/prep

env: BASE_DIR=/home/jupyter/prep


In [53]:
!gsutil cp gs://cs327e-open-access/bq_college.zip $BASE_DIR

Copying gs://cs327e-open-access/bq_college.zip...
/ [1 files][  1.9 KiB/  1.9 KiB]                                                
Operation completed over 1 objects/1.9 KiB.                                      


In [54]:
!unzip $BASE_DIR/bq_college.zip

Archive:  /home/jupyter/prep/bq_college.zip
   creating: bq_college/
  inflating: bq_college/takes.csv    
  inflating: bq_college/teaches.csv  
  inflating: bq_college/instructor.csv  
  inflating: bq_college/class.csv    
  inflating: bq_college/student.csv  


#### 2. Load college data into BQ tables:
##### Please refer to https://cloud.google.com/bigquery/docs/reference/bq-cli-reference#top_of_page for documentation on the bq tool its `bq load` command

In [26]:
%env dataset_id=college

env: dataset_id=college


In [29]:
!bq --location=US mk --dataset $dataset_id

BigQuery error in mk operation: Dataset 'cs327e-sp2022:college' already exists.


In [58]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV ${dataset_id}.Class ${BASE_DIR}/bq_college/class.csv

Upload complete.
Waiting on bqjob_r400fc204684c6bcd_0000017f32732204_1 ... (1s) Current status: DONE   


In [21]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV ${dataset_id}.Student ${BASE_DIR}/bq_college/student.csv

Upload complete.
Waiting on bqjob_r82063cd3785a2bf_0000017f32505f10_1 ... (1s) Current status: DONE   


In [22]:
!bq --location=US load --skip_leading_rows=1 --allow_jagged_rows=true --source_format=CSV ${dataset_id}.Instructor \
${BASE_DIR}/bq_college/instructor.csv \
tid:STRING,name:STRING,dept:STRING

Upload complete.
Waiting on bqjob_r2d940e99bb3b8763_0000017f32508001_1 ... (1s) Current status: DONE   


In [23]:
!bq --location=US load --skip_leading_rows=1 --allow_jagged_rows=true --source_format=CSV ${dataset_id}.Takes \
${BASE_DIR}/bq_college/takes.csv \
sid:STRING,cno:STRING,grade:STRING

Upload complete.
Waiting on bqjob_r16ef76071e04c5c4_0000017f3250936b_1 ... (1s) Current status: DONE   


In [24]:
!bq --location=US load --skip_leading_rows=1 --allow_jagged_rows=true --source_format=CSV ${dataset_id}.Teaches \
${BASE_DIR}/bq_college/teaches.csv \
tid:STRING,cno:STRING

Upload complete.
Waiting on bqjob_r1bd8c36aba4b9c99_0000017f3250a702_1 ... (1s) Current status: DONE   


#### 3. Describe the tables:

In [30]:
!bq show --schema=true $dataset_id.Student

[{"name":"sid","type":"STRING","mode":"NULLABLE"},{"name":"fname","type":"STRING","mode":"NULLABLE"},{"name":"lname","type":"STRING","mode":"NULLABLE"},{"name":"dob","type":"DATE","mode":"NULLABLE"},{"name":"status","type":"STRING","mode":"NULLABLE"}]


In [31]:
!bq show --schema=true $dataset_id.Instructor

[{"name":"tid","type":"STRING","mode":"NULLABLE"},{"name":"name","type":"STRING","mode":"NULLABLE"},{"name":"dept","type":"STRING","mode":"NULLABLE"}]


In [56]:
!bq show --schema=true $dataset_id.Class

[{"name":"cno","type":"STRING","mode":"NULLABLE"},{"name":"cname","type":"STRING","mode":"NULLABLE"},{"name":"credits","type":"INTEGER","mode":"NULLABLE"}]


In [33]:
!bq show --schema=true $dataset_id.Takes

[{"name":"sid","type":"STRING","mode":"NULLABLE"},{"name":"cno","type":"STRING","mode":"NULLABLE"},{"name":"grade","type":"STRING","mode":"NULLABLE"}]


In [34]:
!bq show --schema=true $dataset_id.Teaches

[{"name":"tid","type":"STRING","mode":"NULLABLE"},{"name":"cno","type":"STRING","mode":"NULLABLE"}]


#### 4. Get the record counts:

In [36]:
%%bigquery
select count(*) as count from college.Student

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.07rows/s]


,count
0,12


In [59]:
%%bigquery
select count(*) as count from college.Class

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.05rows/s]


,count
0,10


In [38]:
%%bigquery
select count(*) as count from college.Instructor

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.03s/rows]


,count
0,10


In [39]:
%%bigquery
select count(*) as count from college.Takes

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.02rows/s]


,count
0,16


In [40]:
%%bigquery
select count(*) as count from college.Teaches

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.07s/rows]


,count
0,12


#### 5. Verify the loads by retrieving a few records from each table:

In [60]:
%%bigquery
select * from college.Class
limit 10

Downloading: 100%|██████████| 10/10 [00:00<00:00, 10.21rows/s]


,cno,cname,credits
0,CS329E,Elements of Web Programming,3
1,CS327E,Elements of Databases,3
2,CS313E,Elements of Software Engineering,3
3,CS326E,Elements of Networking,3
4,CS439,Principles of Computer Systems,3
5,M358K,Applied Statistics,3
6,CS347,Data Management,3
7,M362K,Probability I,3
8,M328K,Intro to Number Theory,3
9,CS303E,Elements of Computers and Programming,3


In [61]:
%%bigquery
select * from college.Student
limit 10

Downloading: 100%|██████████| 10/10 [00:00<00:00, 10.03rows/s]


,sid,fname,lname,dob,status
0,kev18,Kevin,Lin,1999-10-05,CUR
1,sudeepa4,Sudeepa,Roy,2001-10-01,CUR
2,jerryh,Jerry,Hargrove,1999-01-03,CUR
3,jc,James,Cowe,2000-04-22,CUR
4,paulg,Paul,Gore,2000-09-17,CUR
5,aprilz,April,Lopez,2000-10-01,CUR
6,bzen26,Biswa,Zen,1998-04-22,CUR
7,natea,Nathan,Anderson,2000-04-04,PRO
8,kgomez9,Katie,Gomez,2001-03-03,PRO
9,dchen,David,Chen,2000-08-22,PRO


In [62]:
%%bigquery
select * from college.Instructor
limit 10

Downloading: 100%|██████████| 10/10 [00:00<00:00, 10.32rows/s]


,tid,name,dept
0,neeman,Joe Neeman,Mathematics
1,koch,Hans Koch,Mathematics
2,mueller,Peter Mueller,Mathematics
3,tran,Ngoc Tran,Mathematics
4,scohen,Shirley Cohen,Computer Science
5,bulko,Bill Bulko,Computer Science
6,fares,Fares Fraij,Computer Science
7,cannata,Phil Cannata,Computer Science
8,downing,Glenn Downing,Computer Science
9,mitra,Shyamal Mitra,Computer Science


In [63]:
%%bigquery
select * from college.Takes
limit 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  9.91rows/s]


,sid,cno,grade
0,jc,CS313E,None
1,kev18,CS329E,A
2,kev18,M362K,A
3,paulg,CS313E,A
4,paulg,CS329E,A
5,jc,CS327E,B
6,sudeepa4,M328K,B
7,sudeepa4,CS303E,B
8,jc,CS331E,A-
9,paulg,CS326E,A-


In [64]:
%%bigquery
select * from college.Teaches
limit 10

Downloading: 100%|██████████| 10/10 [00:00<00:00, 11.06rows/s]


,tid,cno
0,koch,M328K
1,tran,M358K
2,bulko,CS303E
3,fares,CS331E
4,mitra,CS329E
5,mitra,CS313E
6,neeman,M362K
7,scohen,CS327E
8,cannata,CS326E
9,cannata,CS347


#### Prompt 1: For each class in the database, obtain the number of students taking the class. Return the cno for the class along with the enrollment count. Order the results by the enrollment count in descending order. 

In [65]:
%%bigquery
select c.*, t.*
from college.Class c left join college.Takes t 
on c.cno = t.cno
order by c.cno

Downloading: 100%|██████████| 15/15 [00:00<00:00, 15.93rows/s]


,cno,cname,credits,sid,cno_1,grade
0,CS303E,Elements of Computers and Programming,3,sudeepa4,CS303E,B
1,CS313E,Elements of Software Engineering,3,jc,CS313E,None
2,CS313E,Elements of Software Engineering,3,paulg,CS313E,A
3,CS313E,Elements of Software Engineering,3,bzen26,CS313E,B+
4,CS326E,Elements of Networking,3,paulg,CS326E,A-
5,CS327E,Elements of Databases,3,jc,CS327E,B
6,CS327E,Elements of Databases,3,jerryh,CS327E,B
7,CS329E,Elements of Web Programming,3,kev18,CS329E,A
8,CS329E,Elements of Web Programming,3,paulg,CS329E,A
9,CS329E,Elements of Web Programming,3,jerryh,CS329E,A-


In [68]:
%%bigquery
select c.cno, count(t.sid) as num_students
from college.Class c left join college.Takes t 
on c.cno = t.cno
group by c.cno
order by num_students desc

Downloading: 100%|██████████| 10/10 [00:00<00:00, 10.73rows/s]


,cno,num_students
0,CS329E,3
1,CS313E,3
2,CS327E,2
3,CS326E,1
4,M358K,1
5,CS347,1
6,M362K,1
7,M328K,1
8,CS303E,1
9,CS439,0


#### Prompt 2: For each class in the database in which at least two students are enrolled, how many students are taking the class? Return the cno for the class along with the enrollment count. Order the results by the enrollment count in descending order. 

In [69]:
%%bigquery
select cno, count(*) as num_students
from college.Takes 
group by cno
having count(*) >= 2
order by num_students desc

Downloading: 100%|██████████| 3/3 [00:00<00:00,  3.26rows/s]


,cno,num_students
0,CS313E,3
1,CS329E,3
2,CS327E,2
